# Computer vision


## Table of Contents

* [1. Yolo v8 Introduction](#chapter1)
* [2. Application of Yolov8 on the thymio](#chapter2)
    * [2.1 Images of System Detection](#section_2_1)  
* [3. Detection function](#chapter3)
  * [3.1 Packages required](#section_3_1)
  * [3.2 Explanation of the function](#section_3_2)
* [4. Annotation function](#chapter4) 

## 1. YOLO v8 Introduction <a class="anchor" id="chapter1"></a>



YOLOv8 is the newest state-of-the-art YOLO model that can be used for object detection, image classification, and instance segmentation tasks. YOLOv8 was developed by Ultralytics, who also created the influential and industry-defining YOLOv5 model. YOLOv8 includes numerous architectural and developer experience changes and improvements over YOLOv5.

Yolov8 is based on a Convolutional neural network, the main idea is to train a model using this CNN with an adapted dataset.


Convolutional neural network (CNN) is a regularized type of feed-forward neural network that learns feature engineering by itself via filters (or kernel) optimization. Vanishing gradients and exploding gradients, seen during backpropagation in earlier neural networks, are prevented by using regularized weights over fewer connections.For example, for each neuron in the fully-connected layer 10,000 weights would be required for processing an image sized 100 × 100 pixels.

Sources :

https://blog.roboflow.com/whats-new-in-yolov8/

https://en.wikipedia.org/wiki/Convolutional_neural_network




## 2. Application of Yolov8 on the thymio <a class="anchor" id="chapter2"></a>


The basic idea was to use yolov8 to detect the robot, start point and obstacles at any point in time. We first train our model with :
- annotation of a dataset on Robotflow (https://app.roboflow.com/gabriel-paffi)
- training our system on a google colab to accelare our training process (https://colab.research.google.com/github/roboflow-ai/notebooks/blob/main/notebooks/train-yolov8-object-detection-on-custom-dataset.ipynb#scrollTo=I4bpUIibcV1l)

Once we have obtained our model trained on our dataset, we can use this model to predict the class, position and confidence of any object of our datasets.
  

### 2.1 Images of System Detection <a class="anchor" id="section_2_1"></a>

<div style="text-align:center">
    <img src="./cv_images/robot.png" alt="System detection objects">
</div>

<div style="text-align:center">
    <img src="./cv_images/System.png" alt="System detection objects">
</div>



## 3. Detection function <a class="anchor" id="chapter3"></a>


### 3.1 Packages required <a class="anchor" id="section_3_1"></a>

- open cv2 : https://opencv.org/
- Yolov8 from ultralytics : https://github.com/ultralytics/ultralytics

### 3.2 Explanation of the function <a class="anchor" id="section_3_2"></a>

This function take into input a frame and return :
- an array containing the different object as follow : (class, x_center, y_center, width_object, height_object)
- a frame annotated with the class object on it and the confidence accord to the object

This function use our train model (robotv12.pt) and a minimum confidence to detect an object of 0.55 (1 is maximum confidence).




In [7]:
import cv2
from ultralytics import YOLO
model = YOLO("robotv12.pt")
class_object = ["blackholes","earth","mars", "robot" ]

def detection(frame):



    results = model.track(frame, conf = 0.55, verbose = False)

    annotated_frame = results[0].plot()


    array_box_dim_tot = []
           
   

    for objects in results:
    
            number_objects = len(objects.boxes.cls)
            box_dimensions = objects.boxes.xywhn # normalised dimensions to frame (value of x,y, width and height between 0 and 1)
            for i in range(number_objects) :
              
                array_box_dim_object = []
                confidence = round(objects.boxes.conf[i].item(),3)
                cls = objects.boxes.cls[i].item()
                array_box_dim = box_dimensions[i].tolist() # center x,y width, height
                for i in range(len(array_box_dim)):
                    array_box_dim[i] = round(array_box_dim[i],2)
                array_box_dim_object = [class_object[int(cls)], array_box_dim, confidence]

                array_box_dim_tot.append(array_box_dim_object)


                        

    return(array_box_dim_tot, annotated_frame)


## 4. Annotation function <a class="anchor" id="chapter4"></a>

This function goal is to add the path of the robot on the frame.
The input of this function are :
- a frame
- coordinates : the coordinates of each point from the robot towards the goal

The output is the frame with segment between each point of the path.




In [8]:
def annotation(frame, coord):
    
    for k in range(len(coord)-1):
        
        
        frame = cv2.circle(frame, coord[k], circle_radius, blue,circle_thicknes)
        frame = cv2.line(frame, coord[k], coord[k+1], blue, line_thickness)  
        
        

    
    return frame